In [1]:
import numpy as np
import pandas as pd

##### Assumptions

- Test period should end on 2017-02-11: 90 days before and 90 days after

##### Read in the Data and Filter

In [2]:
products = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/products_of_interest.csv", header=None)
stores = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/store_attributes.csv", usecols=list(range(1, 7)))
transactions = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/transactions.csv", usecols=list(range(1, 6)))

treatment_stores = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/q3_treatment_stores.csv", header=None, names=["treatment"])
control_stores = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/q3_control_stores.csv", header=None, names=["control"])
products_of_interest = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/products_of_interest.csv", header=None, names=["products"])

In [3]:
# Convert to datetime
transactions['date_week'] = pd.to_datetime(transactions['date_week'])

In [4]:
# Check relevance of column
transactions['currency_code'].unique()

# Drop column
transactions.drop(columns=['currency_code'], inplace=True)

In [5]:
# Identify stores of interest
stores_of_interest = pd.concat([treatment_stores["treatment"], control_stores["control"]]).values

# Define dates of interest
pre_start, pre_stop = np.datetime64('2016-07-17'), np.datetime64('2016-10-09')
post_start, post_stop = np.datetime64('2016-11-13'), np.datetime64('2017-02-05')

# Filter transactions
pre_transactions = transactions[transactions['store_id'].isin(stores_of_interest) &
                                transactions['date_week'].between(pre_start, pre_stop) &
                                transactions['product_id'].isin(products_of_interest['products'])]

post_transactions = transactions[transactions['store_id'].isin(stores_of_interest) &
                                transactions['date_week'].between(post_start, post_stop) &
                                transactions['product_id'].isin(products_of_interest['products'])]
                               

In [ ]:
# Label Records
pre_transactions.loc[pre_transactions['store_id'].isin(treatment_stores['treatment']), 'group'] = 'treatment'
pre_transactions.loc[pre_transactions['store_id'].isin(control_stores['control']), 'group'] = 'control'

post_transactions.loc[post_transactions['store_id'].isin(treatment_stores['treatment']), 'group'] = 'treatment'
post_transactions.loc[post_transactions['store_id'].isin(control_stores['control']), 'group'] = 'control'